In [1]:
import praw
import pandas as pd
from datetime import datetime
import re
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
import spacy

In [2]:
reddit = praw.Reddit(client_id='57sReOENpor__WmnpdQAFw', client_secret='4i6Xk64hQI7haFKA2JKBDfOjA2L1Vg', user_agent='WebScraping')

In [3]:
ml_subreddit = reddit.subreddit('bitcoin')

# Retrieve posts from the subreddit
posts = ml_subreddit.new(limit=200)  

# Create a list to store the post data
post_data = []

# Iterate over the posts and extract relevant information
for post in posts:
    post_data.append({
        'Titles': post.title,
    })

# Create a DataFrame from the post data
posts_df = pd.DataFrame(post_data)

# Print the resulting DataFrame
posts_df

,Titles
0,What is the difference between the liquid side...
1,Why are QR codes of the same address not ident...
2,RIP Charlie (rat poison) Munger
3,Charlie Munger dies at age 99
4,Self Custodial Lightning Wallets
...,...
195,What do you do if your wallet 10x’s and is now...
196,Could use a little advice :)
197,Bitcoin mining is the best way to accumulate s...
198,Bitcoin (COIN: $BTC) Reaches New 52-Week Highs


In [4]:
#remove duplicates
posts_df.Titles.duplicated().sum()

0

In [5]:
#Create a function to clean the tweets
def cleanTxt(text):
 text = re.sub(r'@[A-Za-z0–9]+', '', text) 
 text = re.sub(r'#', '', text) 
 text = re.sub(r'RT[\s]+', '', text) 
 text = re.sub(r'https?:\/\/\S+', '', text) #Remove the hyperlinks
 text = re.sub(r':', '', text) # Remove :
 return text
#Cleaning the text
posts_df["Titles"]= posts_df["Titles"].apply(cleanTxt)
#Show the clean text
posts_df.head()

,Titles
0,What is the difference between the liquid side...
1,Why are QR codes of the same address not ident...
2,RIP Charlie (rat poison) Munger
3,Charlie Munger dies at age 99
4,Self Custodial Lightning Wallets


In [6]:
def remove_emoji(string):
     emoji_pattern = re.compile("["
     u"\U0001F600-\U0001F64F" # emoticons
     u"\U0001F300-\U0001F5FF" # symbols & pictographs
     u"\U0001F680-\U0001F6FF" # transport & map symbols
     u"\U0001F1E0-\U0001F1FF" # flags (iOS)
     u"\U00002500-\U00002BEF" # chinese char
     u"\U00002702-\U000027B0"
     u"\U00002702-\U000027B0"
     u"\U000024C2-\U0001F251"
     u"\U0001f926-\U0001f937"
     u"\U00010000-\U0010ffff"
     u"\u2640-\u2642"
     u"\u2600-\u2B55"
     u"\u200d"
     u"\u23cf"
     u"\u23e9"
     u"\u231a"
     u"\ufe0f" # dingbats
     u"\u3030"
     "]+", flags=re.UNICODE)
     return emoji_pattern.sub(r"", string)
#Cleaning the text
posts_df["Titles"]= posts_df["Titles"].apply(remove_emoji)
#Show the clean text
posts_df.head()

,Titles
0,What is the difference between the liquid side...
1,Why are QR codes of the same address not ident...
2,RIP Charlie (rat poison) Munger
3,Charlie Munger dies at age 99
4,Self Custodial Lightning Wallets


In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
# Function to get entities using spaCy
def get_entities(title):
    doc = nlp(title)
    return [ent.text for ent in doc.ents]

# Apply the function to the 'Title' column
posts_df['Entities'] = posts_df['Titles'].apply(get_entities)

In [9]:
# Function to get only sentiment from TextBlob
def get_textblob_sentiment(title):
    analysis = TextBlob(title)
    return 'Positive' if analysis.sentiment.polarity > 0 else 'Negative' if analysis.sentiment.polarity < 0 else 'Neutral'

# Apply the function to the 'Title' column
posts_df['Sentiment'] = posts_df['Titles'].apply(get_textblob_sentiment)

In [10]:
posts_df.head()

,Titles,Entities,Sentiment
0,What is the difference between the liquid side...,[],Neutral
1,Why are QR codes of the same address not ident...,[],Neutral
2,RIP Charlie (rat poison) Munger,"[RIP Charlie, Munger]",Neutral
3,Charlie Munger dies at age 99,"[Charlie Munger, age 99]",Neutral
4,Self Custodial Lightning Wallets,[],Neutral


In [11]:
# Flatten the 'Entities' lists and create a new DataFrame with entity counts
entities_df = pd.DataFrame(posts_df['Entities'].explode().value_counts().reset_index())
entities_df.columns = ['Entity', 'Count']

# Display the entity counts DataFrame
print(entities_df)

                                                Entity  Count
0                                              Bitcoin     22
1                                                first      4
2                                             Coldcard      3
3                                         Bitcoin Core      2
4                                                  one      2
..                                                 ...    ...
129                      Bitcoin Is Useless" Is a Fact      1
130                                     Markus Thielen      1
131  Bitcoin’s Resurgence Fueled by Anticipation of...      1
132                                               CBDC      1
133                             half a million dollars      1

[134 rows x 2 columns]


In [12]:
posts_df.to_csv("bitcoin_reddit.csv",index=False)

In [13]:
entities_df.to_csv("entities_reddit.csv",index=False)